In [1]:
#Import Dependencies
import pandas as pd
import os
import csv

In [2]:
#Set variables for the input file
file_to_load="Resources/purchase_data.csv"
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [3]:
#Count the total first, then use the drop_duplicate to verify if there is any duplicate values
purchase_data.count()

Purchase ID    780
SN             780
Age            780
Gender         780
Item ID        780
Item Name      780
Price          780
dtype: int64

In [4]:
#Find the total number of Players, which is the count of SN column, but there maybe some duplicate values
#so I will try to verify.
cleaned_data_df=purchase_data.drop_duplicates("SN")
cleaned_data_df.count()

Purchase ID    576
SN             576
Age            576
Gender         576
Item ID        576
Item Name      576
Price          576
dtype: int64

In [5]:
#Found that there is one SN id that represented more than one, so the total number players would be the lenght of the SN column
total_players_df=pd.DataFrame([{"Total Players":len(cleaned_data_df['SN'].value_counts())}])
total_players_df

,Total Players
0,576


In [6]:
#Purchasing Analysis
Purchasing_Analysis_df=pd.DataFrame([
    {"Number of Unique Items":len(purchase_data['Item Name'].unique()),
    "Average Price":round(purchase_data['Price'].mean(),2),
    "Number of Purchases":len(purchase_data['Purchase ID']),
    "Total Revenue":purchase_data['Price'].sum()}])
Purchasing_Analysis_df["Average Price"]=Purchasing_Analysis_df["Average Price"].astype(float).map("${:,.2f}".format)
Purchasing_Analysis_df["Total Revenue"]=Purchasing_Analysis_df["Total Revenue"].astype(float).map("${:,.2f}".format)
Purchasing_Analysis_df.head()


,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [7]:
#Gender Demographics
#Since there is some players represented more than one, so the gender would have the duplicate values
#so I would count the gender with the cleaned_data_df
gender_df = pd.DataFrame(cleaned_data_df["Gender"].value_counts())

gender_df.columns = ["Total Count"]

gender_df["Percentage of Players"]=round((gender_df['Total Count']/len(cleaned_data_df['SN'].value_counts()))*100,2)

gender_df["Percentage of Players"]=gender_df["Percentage of Players"].astype(float).map("{0:.2f}%".format)

gender_df.index.name="Gender"

gender_df.head()




,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [8]:
#Purchasing Analysis based on Gender
#I found that when I use groupby function, I can create a new DataFrame, and can use its columns for the analysis

purchase_demographic=purchase_data.groupby("Gender")

purchase_count=purchase_demographic["Purchase ID"].count()

average_purchase=round(purchase_demographic["Price"].mean(),2)

total_purchase=round(purchase_demographic["Price"].sum(),2)

avg_per_person=(purchase_demographic["Price"].sum()/purchase_demographic["SN"].nunique()).round(2)

purchase_demographic_df=pd.DataFrame(
    {"Purchase Count":purchase_count,
     "Average Purchase Price":average_purchase.astype(float).map("${:,.2f}".format),
     "Total Purchase Value":total_purchase.astype(float).map("${:,.2f}".format),
     "Avg Total Purchase per Person":avg_per_person.astype(float).map("${:,.2f}".format)})
purchase_demographic_df.index.name="Gender"
purchase_demographic_df.head()


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [9]:
#Age Demographics, create the bins and assign the name
age_bins = [0, 9.90, 14.90, 19.90, 24.90, 29.90, 34.90, 39.90, 99]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

purchase_data["Age Group"]=pd.cut(purchase_data["Age"],age_bins,labels=group_names)

purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Age Group
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53,20-24
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56,40+
2,2,Ithergue48,24,Male,92,Final Critic,4.88,20-24
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27,20-24
4,4,Iskosia90,23,Male,131,Fury,1.44,20-24


In [10]:
#Using nunique to count the distinct value in SN group, as the total players.
age_df=purchase_data.groupby("Age Group")
age_count=age_df["SN"].nunique()
age_percentage=(age_count/len(cleaned_data_df['SN'].value_counts()))*100
age_demographic_df=pd.DataFrame({
    "Total Count":age_count,
    "Percentage of Players":age_percentage.round(2).map("{0:.2f}%".format),
})
age_demographic_df



,Total Count,Percentage of Players
Age Group,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [11]:
#Purchasing analysis based on age
age_purchase_count=age_df["Purchase ID"].count()

age_avg_price=age_df["Price"].mean()

age_total_purchase=age_df["Price"].sum()

age_average_purchase=(age_total_purchase/age_count)

age_demographic_df=pd.DataFrame({
    "Purchase Count":age_purchase_count,
    "Average Purchase Price":age_avg_price.map("${:,.2f}".format),
    "Total Purchase Value":age_total_purchase.map("${:,.2f}".format),
    "Avg Total Purchase per Person":age_average_purchase.map("${:,.2f}".format),
})
age_demographic_df








,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Group,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [12]:
#Top Spender
#Notes: sort_values will provide false sorting with the formatted values
#Always do the formation after sorting.
spender_stats=purchase_data.groupby("SN")

top_purchase_count=spender_stats["Purchase ID"].count()

ave_purchase_price=spender_stats["Price"].mean()

total_purchase_value=spender_stats["Price"].sum()

top_spender_df=pd.DataFrame({
    "Purchase Count":top_purchase_count,
    "Average Purchase Price":ave_purchase_price,
    "Total Purchase Value":total_purchase_value
})
formatted_df=top_spender_df.sort_values(["Total Purchase Value"],ascending=False).head()
formatted_df.style.format({"Average Purchase Price":"${:,.2f}", 
                            "Total Purchase Value":"${:,.2f}"})


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [13]:
#Most popular item, set new variables, and retrieve the Item ID, Item Name and Price
popular_df=purchase_data[["Item ID","Item Name","Price"]]

popular_stats=popular_df.groupby(["Item ID","Item Name"])
#Count based on item name that appeared on the dataset
popular_count=popular_stats["Item Name"].count()

popular_total=popular_stats["Price"].sum()

popular_price=popular_total/popular_count

popular_df=pd.DataFrame({
    "Purchase Count":popular_count,
    "Item Price":popular_price,
    "Total Purchase Value":popular_total,
})
popular_formatted=popular_df.sort_values("Purchase Count",ascending=False).head()
popular_formatted.style.format({"Item Price":"${:,.2f}", 
                            "Total Purchase Value":"${:,.2f}"})



,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


In [14]:
#Most Profitable Items, using the above table and sort by Total Purchase Value column
popular_formatted=popular_df.sort_values("Total Purchase Value",ascending=False).head()
popular_formatted.style.format({"Item Price":"${:,.2f}", 
                            "Total Purchase Value":"${:,.2f}"})






,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
